In [2]:
from gpad_data_generation import *
from data_loader import *
import sounddevice as sd
import numpy as np

# To parse the xml
file = "extendedballroom_v1.1.xml"

import xml.etree.ElementTree as ET
tree = ET.parse(file)
root = tree.getroot()
# To mesure the running time
import time
# To store the data
import h5py

In [3]:
# xml parsing
start = time.time()
i = 0
i_break = -1
for elem in root:
    # Here we collect the name of the directory
    directory = str(elem.tag) + "/"
    # print(directory)
    for subelem in elem:
        # Here we collect the name of file
        music = subelem.attrib['id'] + ".mp3"
        music_name = subelem.attrib['title']        
        bbpm = int(subelem.attrib['bpm'])
        filename = directory + music
        
        try:
            # Here we get the ODFs, PeDFs and coeffs
            ODF_SET, PeDF_FULL_SET, PeDF_PARTIAL_SET, coeffs = music_processor(filename, start_sample=2**19, final_sample=2**20, sample_rate=44100, nivel_wavelet = 2)

            # Opening the hdf5 file
            hf = h5py.File('h5/n2data_extendedballroom_' + str(i) + '.hdf5','a')

            # Storing Arrays
            ODF = hf.create_group('ODF')
            PeDFfull = hf.create_group('PeDFfull')
            PeDFpartial = hf.create_group('PeDFpartial')
            #coeffswav = hf.create_group('coeffswav')

            for j in range(0,2):
                ODF.create_dataset("ODF" + str(j), data=ODF_SET[j])
                PeDFfull.create_dataset("PeDFfull" + str(j), data=PeDF_FULL_SET[j])
                PeDFpartial.create_dataset("PeDFpartial" + str(j), data=PeDF_PARTIAL_SET[j])
                #coeffswav.create_dataset("coeffswav" + str(j), data=coeffs[j])

            bbbpm = np.array((1,1))
            bbbpm = bbpm
            bpm = hf.create_group('bpm')
            bpm.create_dataset("bpm", data=bbbpm)

            # Storing Strings
            # TODO - music / genre / title

            # Closing the hdf5 file
            hf.close()
        except:
            continue
        
        # Shows
        if i%1 == 0:
            print(f"We've already made {i} calculations.")

        i = i + 1
        if i == i_break:
            break
    if i == i_break:
        break
        
end = time.time()
print(f"O tempo de execução foi de {end - start} segundos")

O tempo de execução foi de 0.17245817184448242 segundos
